<a href="https://colab.research.google.com/github/fpseverino/progetto-ml-ai/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [963]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd

In [964]:
# Carica DataFrame da csv
df = pd.read_csv("Task_21.csv")
df.shape

(145, 134)

In [965]:
# Controlla se ci sono righe duplicate prima di rimuovere l'Id
df.duplicated().sum()

0

In [966]:
# Cancella le colonne "Task" e "Id"
del df["Task"]
del df["Id"]
df.shape

(145, 132)

In [967]:
# Rimuove righe con label uguale a -1
df = df[df["Label"] != -1]
df.shape

(119, 132)

In [968]:
# Controlla se ci sono righe duplicate dopo aver rimosso l'Id
df.duplicated().sum()

1

In [969]:
# Controlla se ci sono valori null
df.isnull().sum().sum()

0

In [970]:
# Controlla se ci sono valori NaN
df.isna().sum().sum()

0

In [971]:
# Controlla tipi di dato delle feature
print(df.dtypes)
print("-------------------------------------------")
df.dtypes.value_counts(dropna = False)

mean:velocity:axis-x(in-air)        float64
std:velocity:axis-x(in-air)         float64
iqr:velocity:axis-x(in-air)         float64
mean:velocity:axis-x(on-surface)    float64
std:velocity:axis-x(on-surface)     float64
                                     ...   
number_of_interruptions_relative    float64
Gender                               object
Age                                 float64
Dominant_Hand                        object
Label                                 int64
Length: 132, dtype: object
-------------------------------------------


float64    129
object       2
int64        1
Name: count, dtype: int64

In [972]:
# La feature Age non ha ne valori pari a 0 ne outlier
df["Age"].value_counts(dropna = False)

Age
6.0    91
7.0    24
9.0     3
8.0     1
Name: count, dtype: int64

In [973]:
# Seleziona le feature numeriche (meno Label e Age)
colonne_numeriche = df.select_dtypes(include=[np.number]).columns
colonne_numeriche = colonne_numeriche.drop("Label")
colonne_numeriche = colonne_numeriche.drop("Age") # La feature Age non ha ne valori pari a 0 ne outlier

In [974]:
# Sostituisce tutti i valori numerici pari a 0 con la media della Label

# Per ogni colonna numerica
for colonna in colonne_numeriche:
    # Calcola la media per ogni Label, escludendo i valori 0
    valori_medi = df[df[colonna] != 0].groupby('Label')[colonna].mean()

    # Per ogni 'Label' nel DataFrame
    for label in df['Label'].unique():
        # Se la Label esiste in valori_medi
        if label in valori_medi.index:
            # Sostituisci i valori 0 con la media corrispondente per quella Label
            df.loc[(df['Label'] == label) & (df[colonna] == 0), colonna] = valori_medi[label]

In [975]:
df.describe()

,mean:velocity:axis-x(in-air),std:velocity:axis-x(in-air),iqr:velocity:axis-x(in-air),mean:velocity:axis-x(on-surface),std:velocity:axis-x(on-surface),iqr:velocity:axis-x(on-surface),mean:velocity:axis-y(in-air),std:velocity:axis-y(in-air),iqr:velocity:axis-y(in-air),mean:velocity:axis-y(on-surface),...,mean:ratio_of_stroke_durations,std:ratio_of_stroke_durations,iqr:ratio_of_stroke_durations,writing_duration(on-surface),writing_duration_overall,ratio_of_writing_durations,number_of_interruptions,number_of_interruptions_relative,Age,Label
count,119.000000,119.000000,119.000000,119.000000,119.000000,1.190000e+02,119.000000,119.000000,119.000000,119.000000,...,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
mean,1.092989,6.637259,2.178910,0.591317,0.913416,2.000000e+00,1.366033,8.214521,1.980917,0.686141,...,17.629575,11.137034,11.216164,40.102353,41.844538,48.090299,8.264706,0.169823,6.294118,1.890756
std,0.902524,8.882468,0.402854,0.235574,0.140344,3.904450e-13,0.937642,8.527273,0.425609,0.270053,...,13.989354,7.512442,7.684848,12.893768,14.628517,108.135553,6.419258,0.100668,0.615519,0.407346
min,0.161538,0.544961,2.000000,0.235552,0.644685,2.000000e+00,0.219178,0.624754,0.000000,0.276198,...,0.250000,0.010440,0.010440,14.415000,14.420000,3.880740,2.000000,0.033904,6.000000,0.000000
25%,0.770749,3.144829,2.000000,0.437244,0.831900,2.000000e+00,1.032000,4.683982,2.000000,0.506247,...,7.750566,6.919434,6.126454,31.340000,31.410000,23.796485,4.000000,0.113533,6.000000,2.000000
50%,1.112846,6.987453,2.193548,0.540244,0.897260,2.000000e+00,1.430516,8.678108,2.052083,0.639766,...,16.819949,11.710702,11.844956,39.115000,40.160000,36.806636,8.500000,0.173889,6.000000,2.000000
75%,1.112846,6.987453,2.193548,0.662932,0.957372,2.000000e+00,1.430516,8.678108,2.052083,0.763892,...,16.819949,11.710702,11.844956,48.200000,50.247500,36.806636,8.500000,0.173889,6.000000,2.000000
max,9.382353,77.355247,6.000000,1.585885,1.801404,2.000000e+00,9.970588,82.580318,4.000000,1.930026,...,97.868687,54.992345,58.778139,91.275000,115.030000,1105.375000,46.000000,0.586611,9.000000,2.000000


In [976]:
# Seleziona solo le colonne numeriche (meno Label e Age)
df_numeric = df[colonne_numeriche]

# Calcola i percentili per ciascuna colonna numerica
q_low = df_numeric.quantile(0.01)
q_high = df_numeric.quantile(0.99)

# Calcola la media per ciascuna colonna per ogni label
mean_per_label = df.groupby('Label')[colonne_numeriche].transform('mean')

# Sostituisci gli outlier con la media della colonna per ogni label
df_filtered = df_numeric.where((df_numeric > q_low) & (df_numeric < q_high), mean_per_label)

# Ora df_filtered contiene solo i valori non outlier nelle colonne numeriche

# Seleziona le colonne non numeriche (più Label e Age)
df_non_numerics = df.select_dtypes(exclude=[np.number])
df_non_numerics = df_non_numerics.join(df["Label"])
df_non_numerics = df_non_numerics.join(df["Age"])

# Unisci i due DataFrame
df = df_filtered.join(df_non_numerics)

In [977]:
df.describe()

,mean:velocity:axis-x(in-air),std:velocity:axis-x(in-air),iqr:velocity:axis-x(in-air),mean:velocity:axis-x(on-surface),std:velocity:axis-x(on-surface),iqr:velocity:axis-x(on-surface),mean:velocity:axis-y(in-air),std:velocity:axis-y(in-air),iqr:velocity:axis-y(in-air),mean:velocity:axis-y(on-surface),...,mean:ratio_of_stroke_durations,std:ratio_of_stroke_durations,iqr:ratio_of_stroke_durations,writing_duration(on-surface),writing_duration_overall,ratio_of_writing_durations,number_of_interruptions,number_of_interruptions_relative,Label,Age
count,119.000000,119.000000,119.000000,119.000000,119.000000,1.190000e+02,119.000000,119.000000,119.000000,119.000000,...,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000,119.000000
mean,1.009306,5.695717,2.136622,0.580946,0.905975,2.000000e+00,1.295308,7.502799,1.956583,0.673238,...,16.759189,10.770081,10.778611,39.823984,41.407057,39.047637,8.831933,0.166030,1.890756,6.294118
std,0.320667,3.082324,0.088562,0.195825,0.101095,3.327737e-13,0.454648,4.494619,0.370516,0.218834,...,9.580913,5.751180,5.518564,11.144116,12.113481,41.633936,4.038101,0.086366,0.407346,0.615519
min,0.267123,0.680361,2.000000,0.313755,0.728887,2.000000e+00,0.363636,0.771389,0.000000,0.374202,...,1.000000,1.085589,1.085589,16.400000,16.405000,4.284649,4.000000,0.035862,0.000000,6.000000
25%,0.819923,3.364900,2.000000,0.444695,0.833713,2.000000e+00,1.053872,4.914071,2.000000,0.518994,...,8.626780,7.419768,6.931653,31.955000,32.232500,25.280855,8.500000,0.116891,2.000000,6.000000
50%,1.112846,6.987453,2.193548,0.549108,0.902133,2.000000e+00,1.430516,8.678108,2.052083,0.649702,...,16.819949,11.710702,11.844956,40.007273,40.805000,36.806636,8.500000,0.173889,2.000000,6.000000
75%,1.112846,6.987453,2.193548,0.654142,0.954804,2.000000e+00,1.430516,8.678108,2.052083,0.752239,...,16.819949,11.710702,11.844956,47.997500,49.447500,36.806636,8.500000,0.173889,2.000000,6.000000
max,2.169049,16.871146,2.193548,1.460284,1.245485,2.000000e+00,2.727554,26.186697,2.052083,1.614984,...,49.487805,34.960985,33.176471,70.720000,74.185000,314.345524,28.000000,0.451249,2.000000,9.000000


In [978]:
# Controllare valori della colonna Gender
print(df['Gender'].value_counts())

# Sostituire le stringhe con valori numerici
df["Gender"] = df.Gender.apply(lambda x: x.replace("Femminile", "0"))
df["Gender"] = df.Gender.apply(lambda x: x.replace("Maschile", "1"))

print("-------------------------")
print(df['Gender'].value_counts())

# I valori rimangono di tipo "object" (stringhe)
print("-------------------------")
print(df.dtypes.value_counts(dropna = False))

Gender
Femminile    64
Maschile     55
Name: count, dtype: int64
-------------------------
Gender
0    64
1    55
Name: count, dtype: int64
-------------------------
float64    129
object       2
int64        1
Name: count, dtype: int64


In [979]:
# Controllare valori della colonna Dominant_Hand
print(df['Dominant_Hand'].value_counts())

# Sostituire le stringhe con valori numerici
df["Dominant_Hand"] = df.Dominant_Hand.apply(lambda x: x.replace("Destra", "0"))
df["Dominant_Hand"] = df.Dominant_Hand.apply(lambda x: x.replace("Sinistra", "1"))

print("-------------------------")
print(df['Dominant_Hand'].value_counts())

# I valori rimangono di tipo "object" (stringhe)
print("-------------------------")
print(df.dtypes.value_counts(dropna = False))

Dominant_Hand
Destra      106
Sinistra     13
Name: count, dtype: int64
-------------------------
Dominant_Hand
0    106
1     13
Name: count, dtype: int64
-------------------------
float64    129
object       2
int64        1
Name: count, dtype: int64


In [980]:
# Controlla se ci sono valori NaN
df.isna().sum().sum()

0

In [981]:
# Controlla se ci sono valori null
df.isnull().sum().sum()

0